In [1]:
import numpy as np
import numpy.linalg
import scipy.io
import meshplot as mp
from igl import *
import meshio
import h5py

import sys
sys.path.append('./ShellUtils')
%load_ext autoreload
%autoreload 2
import Gen, TriMesh, Visual, Geometry

In [10]:
InputH5 = '/Users/ziyizhang/Projects/Zebrafish/data/ctrl_em1_filter-analysis-02_18T09_59-AnalysisInput.h5'
idxArray = np.array([1, 2, 3, 6, 12, 28], dtype=int)  # index of markers to align
                                                  # cannot be too small or too symmetric

In [11]:
# read re-analysis H5 file
with h5py.File(InputH5, 'r+') as f:
    print("Keys: %s" % f.keys())

    E = f['E'].value[()]
    F = f['F'].value[()]
    Nverts = f['Nverts'].value[()]
    V = f['V'].value[()]
    discr_order = f['discr_order'].value[()]
    frames = f['frames'].value[()]
    imgCols = f['imgCols'].value[()]
    imgRows = f['imgRows'].value[()]
    is_linear = f['is_linear'].value[()]
    layerPerImg = f['layerPerImg'].value[()]
    markerRCMap_mat = f['markerRCMap_mat'].value[()]
    max_tet_vol = f['max_tet_vol'].value[()]
    n_refs = f['n_refs'].value[()]
    nu = f['nu'].value[()]
    offset = f['offset'].value[()]
    radius_edge_ratio = f['radius_edge_ratio'].value[()]
    resolutionX = f['resolutionX'].value[()]
    resolutionY = f['resolutionY'].value[()]
    resolutionZ = f['resolutionZ'].value[()]
    upsample = f['upsample'].value[()]
    vismesh_rel_area = f['vismesh_rel_area'].value[()]

    VV = np.zeros((frames, Nverts, 3))
    for i in range(frames):
        VV[i, :, :] = V[i*Nverts:(i+1)*Nverts, :]

Keys: <KeysViewHDF5 ['E', 'F', 'Nverts', 'V', 'discr_order', 'frames', 'imgCols', 'imgRows', 'is_linear', 'layerPerImg', 'markerRCMap_mat', 'max_tet_vol', 'n_refs', 'nu', 'offset', 'radius_edge_ratio', 'resolutionX', 'resolutionY', 'resolutionZ', 'upsample', 'vismesh_rel_area']>


In [18]:
# re-store global displacement
#dispTxt = '/Users/ziyizhang/Projects/tmp/meanDisp.txt'
#meanDisp = np.loadtxt(dispTxt)
#VV_raw = np.array([VV[i, :, :]+meanDisp[i, :].reshape(1, 3) for i in range(VV.shape[0])])
VV_raw = VV

In [19]:
# The area to estimate global displacement
# idxArray = np.array([1, 2, 3, 6, 12], dtype=int)
alignPosArray = np.array([VV_raw[i, idxArray, :] for i in range(VV_raw.shape[0])])

In [20]:
# Visualize the selected index array (before alignment)
# red is frame-0, blue is frame-"frame"
frame = VV_raw.shape[0]-1
plt = mp.plot(VV_raw[0], Geometry.f2e(F))
plt.add_edges(VV_raw[frame], Geometry.f2e(F))
plt.add_points(alignPosArray[0], shading={"point_color": "red", "point_size": 26})
plt.add_points(alignPosArray[frame], shading={"point_color": "blue", "point_size": 20})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(159.10311…

3

In [22]:
# do alignment
VV_new = np.zeros((VV_raw.shape[0], VV_raw.shape[1], VV_raw.shape[2]))
alignPosArray_post = np.zeros((alignPosArray.shape[0], alignPosArray.shape[1], alignPosArray.shape[2]))
for f in range(VV_raw.shape[0]):
    #print(f)
    s, R, T = igl.procrustes(alignPosArray[f], alignPosArray[0], False, False)
    #print(R)
    #print(T)
    assert(s==1.0)
    #tt = np.matmul(VV_raw[0], R) + T.reshape(1, 3)
    #print("    ->   ", np.min(tt - VV_raw[f]))
    VV_new[f, :, :] = np.matmul(VV_raw[f], R) + T.reshape(1, 3)
    alignPosArray_post[f, :, :] = np.matmul(alignPosArray[f], R) + T.reshape(1, 3)

In [23]:
# Visualize the selected index array (after rotation+translation alignment)
# red is frame-0, blue is frame-"frame"
frame = VV_raw.shape[0]-1
plt = mp.plot(VV_raw[0], Geometry.f2e(F))
plt.add_edges(VV_new[frame], Geometry.f2e(F))
plt.add_points(alignPosArray[0], shading={"point_color": "red", "point_size": 26})
plt.add_points(alignPosArray_post[frame], shading={"point_color": "blue", "point_size": 20})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(159.10311…

3

In [24]:
# write result to new H5 file
# read re-analysis H5 file
OutputH5 = InputH5[:-3] + "-NewGlobalDisp.h5"

# get V_new
V_new = VV_new.reshape(Nverts*frames, 3)
VV_ = np.zeros((frames, Nverts, 3))
for i in range(frames):
    VV_[i, :, :] = V_new[i*Nverts:(i+1)*Nverts, :]
assert(np.min(VV_-VV_new) == 0)

with h5py.File(OutputH5, 'w') as f:

    f.create_dataset('E', data=E)
    f.create_dataset('F', data=F)
    f.create_dataset('Nverts', data=Nverts)
    f.create_dataset('V', data=V_new)
    f.create_dataset('discr_order', data=discr_order)
    f.create_dataset('frames', data=frames)
    f.create_dataset('imgCols', data=imgCols)
    f.create_dataset('imgRows', data=imgRows)
    f.create_dataset('is_linear', data=is_linear)
    f.create_dataset('layerPerImg', data=layerPerImg)
    f.create_dataset('markerRCMap_mat', data=markerRCMap_mat)
    f.create_dataset('max_tet_vol', data=max_tet_vol)
    f.create_dataset('n_refs', data=n_refs)
    f.create_dataset('nu', data=nu)
    f.create_dataset('offset', data=offset)
    f.create_dataset('radius_edge_ratio', data=radius_edge_ratio)
    f.create_dataset('resolutionX', data=resolutionX)
    f.create_dataset('resolutionY', data=resolutionY)
    f.create_dataset('resolutionZ', data=resolutionZ)
    f.create_dataset('upsample', data=upsample)
    f.create_dataset('vismesh_rel_area', data=vismesh_rel_area)

    VV = np.zeros((frames, Nverts, 3))
    for i in range(frames):
        VV[i, :, :] = V[i*Nverts:(i+1)*Nverts, :]

In [10]:
VV_new_cache = VV_new

In [ ]:
meanL = np.array([igl.avg_edge_length(np.squeeze(VV[f, :, :]), F) for f in range(frames)])

In [ ]:
F.shape

In [ ]:
tt = np.squeeze(VV[1, :, :])
mp.plot(tt, F, shading={"wireframe": True})

In [ ]:
# Read debug msh
with h5py.File('/Users/ziyizhang/Projects/Zebrafish/release/test.h5', 'r+') as f:
    print("Keys: %s" % f.keys())
    T = f['T'].value[()]
    V = f['V'].value[()]

    bm_v = f['bm_v'].value[()] 
    bm_f = f['bm_f'].value[()] 
    Tid = f['Tid'].value[()]
    result_f = f['result_f'].value[()]

In [ ]:
# clip
z_min = -10
z_max = 30

plt = mp.plot(V, bm_f, shading={"wireframe": True})
V_clip = np.array([V[i, 2] > z_min and V[i, 2] < z_max for i in range(V.shape[0])])
T_clip = T[np.array([V_clip[T[i, 0]] and V_clip[T[i, 1]] and V_clip[T[i, 2]] and V_clip[T[i, 3]] for i in range(T.shape[0])])]
plt.add_edges(V, Geometry.t2e(T_clip))
plt.add_edges(bm_v, Geometry.f2e(bm_f), shading={"line_color": "red"})

In [ ]:
print(Tid.shape)
Tid_unique, count = np.unique(Tid, return_counts=True)
print(Tid_unique.shape)
Tid_unique[count > 1]

In [ ]:
T_boundary = igl.boundary_facets(T)
components = igl.face_components(T_boundary)
print(components.max() + 1)

plt = mp.plot(V, bm_f, shading={"wireframe": True})
#plt = mp.plot(V, Geometry.t2e(T[Tid, :]), shading={"wireframe": True})
plt.add_edges(V, Geometry.t2e(T[Tid]), shading={"line_color": "blue"})
plt.add_edges(V, Geometry.f2e(bm_f), shading={"line_color": "black"})
# plt.add_points(V[[129], :], shading={"point_color": "red", "point_size": 5})

plt.add_points(V[[678]], shading={"point_color": "red", "point_size": 5})
plt.add_points(V[[708]], shading={"point_color": "blue", "point_size": 5})
plt.add_points(V[[147]], shading={"point_color": "blue", "point_size": 5})
plt.add_points(V[[150]], shading={"point_color": "blue", "point_size": 5})

In [ ]:
t_with_129 = np.where(np.any(T[Tid] == 129, axis = 1)) 

In [ ]:
np.unique(Tid).shape

In [ ]:
plt = mp.plot(V, Geometry.t2e(T[Tid][t_with_129]), shading={"wireframe": "true", "line_color": "red"})
plt.add_edges(V, Geometry.f2e(bm_f))

plt.add_points(V[np.array([149, 138, 141, 129])], shading={"point_color": "red", "point_size": 5})
plt.add_points(V[np.array([1679])], shading={"point_color": "blue", "point_size": 5})

In [ ]:
TTid = T[Tid]

In [ ]:
Tid_array = T[Tid].flatten()

In [ ]:
Tid_array = np.sort(Tid_array)

In [ ]:
bmid_array = bm_f.flatten()
bmid_array = np.sort(bmid_array)

In [ ]:
np.where( bmid_array-Tid_array[0:906] != 0 )

In [ ]:
bmid_array[675]

In [ ]:
bmid_array

In [ ]:
np.squeeze(Tid_array[0:906])

In [ ]:
f = h5py.File('/Users/ziyizhang/Projects/Zebrafish/data/em5_19nov2020-3-1-analysis-02_15T02_04-AnalysisInput.h5', 'r+')
print("Keys: %s" % f.keys())
E = f['E'].value[()]
F = f['F'].value[()]
Nverts = f['Nverts'].value[()]
V = f['V'].value[()]
discr_order = f['discr_order'].value[()]
frames = f['frames'].value[()]
markerRCMap_mat = f['markerRCMap_mat'].value[()]

In [ ]:
plt = mp.plot(shell_top, Geometry.f2e(F_h5), shading={"wireframe": True})
plt.add_edges(shell_base, Geometry.f2e(F_h5))
#plt.add_mesh(shell_top, Geometry.f2e(F_h5))

In [ ]:
scipy.io.savemat('./MatlabScripts/length.mat', mdict={'mat': lengthMat})

In [ ]:
# Gen cube test ply
V_cube, F_cube = Gen.GenCubeObj()
igl.write_triangle_mesh("./data/dualShell_cube.ply", V_cube, F_cube, False)

In [ ]:
# dualShell_test
Vraw1 = np.array([[1, 1, 1], [2, 1, 1], [1, 2, 1]])
Vraw2 = np.array([[1, 1, 2], [2, 1, 2], [1, 2, 2]])
Vraw3 = np.array([[1, 1, 3], [3, 1, 3], [1, 3, 3]])
Vraw4 = np.array([[1, 1, 4], [3, 1, 4], [1, 3, 4]])
Fraw = np.array([[0, 1, 2]])
V_shell = np.concatenate((Vraw1, Vraw2), axis=0)
V_shell = np.concatenate((V_shell, Vraw3), axis=0)
V_shell = np.concatenate((V_shell, Vraw4), axis=0)

randTransMat = np.array([[-0.700390490817757,  -0.150045418195264,  -0.697810527901858],
                         [-0.114407726497277,  -0.941413431486692,   0.317256399674106],
                         [-0.704531072763853,   0.302038281505938,   0.642190660174339]])
V_shell = randTransMat.dot(V_shell.transpose()).transpose()

F_shell = np.concatenate((Fraw, Fraw+Vraw1.shape[0]), axis=0)
F_shell = np.concatenate((F_shell, Fraw+2*Vraw1.shape[0]), axis=0)
F_shell = np.concatenate((F_shell, Fraw+3*Vraw1.shape[0]), axis=0)

In [ ]:
# Load processed mesh
# mesh = meshio.read('./dataset/51509.stl.h5_.msh')
# mesh = meshio.read('./dataset/log/log_Dec_26_1000/data/128483.stl.h5_.msh')
# mesh = meshio.read('./dataset/log/log_Dec_27_1000/data/62286.stl.h5_.msh')
# mesh = meshio.read('./dataset/log/log_Dec_28_1000/data/611675.stl.h5_.msh')
# mesh = meshio.read('./dataset/log/log_Dec_28_1000/data/77918.stl.h5_.msh')
# mesh = meshio.read('./dataset/log/log_Dec_28_1000/data/109130.stl.h5_.msh')
# mesh = meshio.read('./dataset/tetgen_result.msh')
# mesh = meshio.read('dataset/cube_2_5.obj.h5_col6.h_.msh')
mesh = meshio.read('dataset/main_linkage_arm_6a_scaled_unpositioned_back_1.hh.sat.obj.h5_col9.h_.msh')
# mesh = meshio.read('./data/dualShell_cube_.msh')
V_msh = mesh.points
T_msh = mesh.cells[0][1]
#energy = mesh.cell_data_dict['slim_energy']['tetra']
 # label = mesh.cell_data['label'][0]

In [ ]:
mp.plot(V_msh, Geometry.t2e(T_msh), shading={"wireframe": True})

In [ ]:
T_boundary = igl.boundary_facets(T_msh)
components = igl.face_components(T_boundary)
print(components.max() + 1)
plt = mp.plot(V_msh, T_boundary[(components == 3), :], shading={"wireframe": True})
plt.add_edges(V_msh, Geometry.f2e(T_boundary[(components == 1) | True, :]), shading={"wireframe": True})

#v = np.array([[-1.6, 1.6, -1.6], [1.6, 1.6, -1.6], [1.6, 1.6, 1.6], [-0.49459213746493375, 1.7758694311449574, 0.39867713464991789], [2.1117691453623983, 2.1117691453623983, -2.1117691453623983]])
#plt.add_mesh(v, (np.array([[0, 1, 2, 3], [0, 1, 2, 4]])), shading={"wireframe": True})

In [ ]:
print(np.nonzero(energy > 10000))
print(np.where(np.abs(energy - 18885) < 3))
print(energy[10235])
print(energy[10236])
print(T_msh[804, :])
print(T_msh[10235, :])
print(T_msh[10236, :])

In [ ]:
len(str)

In [ ]:
V_in, F_in = igl.read_triangle_mesh('/Users/ziyizhang/Projects/ShellMesh/dataset/thin_wall_1.hh.sat.obj.h5.ply')
print(V_in.shape)
print(F_in.shape)
print(F_in.max())

In [ ]:
F_in[984, :]

In [ ]:
a = igl.doublearea(V_in, F_in)

In [ ]:
print(np.max(igl.face_components(F_in)))
igl.boundary_facets(F_in)

In [ ]:
print(V_in.shape)
print(V_msh.shape)
print(F_in.shape)
print(F_boundary.shape)

In [ ]:
F_boundary = igl.boundary_facets(T_msh)
print(np.unique(F_boundary))
vidx = np.setdiff1d(np.arange(1794), np.unique(F_boundary))
print(vidx)

In [ ]:
np.max(F_in)

In [ ]:
plt = mp.plot(V_in, Geometry.f2e(F_in), shading={"wireframe": True})
#plt = mp.plot(V_msh, Geometry.f2e(F_boundary))
#plt = mp.plot(V_msh, T_msh, shading={"wireframe": True})
#plt.add_points(V_msh[[1793], :], shading={"point_size": 0.01})

In [ ]:
tt = np.arange(1400)
vidx = np.setdiff1d(tt, np.unique(F))

In [ ]:
mp.plot(V_msh, Geometry.t2e(T_msh))

In [ ]:
plt = mp.plot(V_msh, Geometry.f2e(F_in), shading={"wireframe": True})
plt.add_points(V_in[vidx, :], shading={"point_size": 0.01, "point_color": "green"})

In [ ]:
# Bad tets
T_msh_bad = T_msh[:, :]
#T_msh_bad = T_msh[[804], :]
print("#bad_tets = {}".format(T_msh_bad.shape[0]))
vol = igl.volume(V_msh, T_msh_bad)
plt = mp.plot(V_msh, T_msh_bad, shading={"wireframe": True})


plt.add_points(V_msh[[T_msh_bad[0, 0]], :], shading={"point_size": 0.01, "point_color": "green"})
plt.add_points(V_msh[[T_msh_bad[0, 1]], :], shading={"point_size": 0.01, "point_color": "yellow"})
plt.add_points(V_msh[[T_msh_bad[0, 2]], :], shading={"point_size": 0.01, "point_color": "black"})
plt.add_points(V_msh[[T_msh_bad[0, 3]], :], shading={"point_size": 0.01})  # red

# Add input shell
V_in, F_in = igl.read_triangle_mesh('/Users/ziyizhang/Projects/ShellMesh/dataset/109130.stl.h5.ply')
B0, B1, T1, T0 = np.split(V_in, 4)
plt.add_lines(B1[Geometry.f2e(F_in)[:, 0]], B1[Geometry.f2e(F_in)[:, 1]], shading={"line_color": "red"})
#plt.add_lines(B0[Geometry.f2e(F_in)[:, 0]], B0[Geometry.f2e(F_in)[:, 1]], shading={"line_color": "red"})
plt.add_lines(T1[Geometry.f2e(F_in)[:, 0]], T1[Geometry.f2e(F_in)[:, 1]], shading={"line_color": "blue"})
#plt.add_mesh(T1, F_in)
#plt.add_edges(B0, Geometry.f2e(F_in), shading={"wireframe": True})
#plt.add_edges(B1, Geometry.f2e(F_in), shading={"wireframe": True})


In [ ]:
# Plot one-ring of a vert
V, F = igl.read_triangle_mesh('/Users/ziyizhang/Projects/ShellMesh/dataset/611675.stl.h5.ply')
B0, B1, T1, T0 = np.split(V, 4)
V1, T1 = igl.read_msh('./dataset/log/log_Dec_28_1000/data/611675.stl.h5_.msh')
targetVert = 2707
partT = T1[ np.where(T1==targetVert)[0] ]
print("#tets = {}".format(partT.shape[0]))

plt = mp.plot(V1, Geometry.t2e(partT))
#plt.add_points(V1[[targetVert]])
plt.add_points(V_msh[[T_msh_bad[0, 0]], :], shading={"point_size": 0.01, "point_color": "green"})
plt.add_points(V_msh[[T_msh_bad[0, 1]], :], shading={"point_size": 0.01, "point_color": "yellow"})
plt.add_points(V_msh[[T_msh_bad[0, 2]], :], shading={"point_size": 0.01, "point_color": "black"})
plt.add_points(V_msh[[T_msh_bad[0, 3]], :], shading={"point_size": 0.01})  # red

In [ ]:
min(1, 2)

In [ ]:
V2 = V1.copy()
V2[11128] = V2[872]
p = mp.plot(V2, Geometry.t2e(partT), wireframe=True)
p.add_points(V2[[11128,872]], point_size=0.02)
p.add_edges(V2, Geometry.f2e(igl.boundary_facets(partT)), line_color='red')
np.linalg.norm(V1[np.unique(partT)] - V1[[11128]],axis=1)[:-1].argmin()
igl.volume(V2, partT)
partT[partT==11128] = 872
igl.volume(V2,np.array([t for t in partT if len(set(t)) ==4]))

In [ ]:
# torch
import torch as th

thV = th.from_numpy(V1)
thV.requires_grad_()
x = tw_mips3d(thV, th.from_numpy(partT))
x.sum().backward()
V2 = V1.copy()
V2[11128] -= 1e-20*thV.grad[11128].numpy()
(tw_mips3d(V2,partT)).sum()
(tw_mips3d(V1,partT)).sum()
igl.volume(V2, partT).min()
igl.volume(V1, partT).min()


def mips3d(V, T):
    from numpy import sqrt
    Jacs = V[T[:,1:]]-V[T[:,:1]]
    invstd = th.from_numpy(np.linalg.inv(np.array([[1,0,0],[0.5,sqrt(3)/2.,0],[0.5,sqrt(3)/6.,sqrt(2)/sqrt(3)]])))
    Jacs = invstd@Jacs
    frob = th.sum(Jacs**2,axis=1).sum(axis=1)
    #invfrob = th.sum(th.inverse(Jacs))
    dets = th.det(Jacs)
    return frob/dets**(2/3)

In [ ]:
V_in, F_in = igl.read_triangle_mesh('/Users/ziyizhang/Projects/ShellMesh/dataset/51509.stl.h5.ply')
plt = mp.plot(V_msh, T_msh[energy > 500, :], c = np.ones(3)/2, shading={"wireframe": True})
B0,B1,T1,T0 = np.split(V_in, 4)
plt.add_mesh(T0, (F_in), shading={"wireframe": True})
plt.add_mesh(T1, (F_in), shading={"wireframe": True})

In [ ]:
vols = igl.volume(V_msh, T_msh)
# B0,B1,T1,T0 = np.split(Vs,4)
np.median(vols)
T_boundary = igl.boundary_facets(T_msh)
components = igl.face_components(T_boundary)
p = mp.plot(V_msh, T_boundary[(components == -1), :], shading={"wireframe": True})
p.add_mesh(V_msh, T_msh[vols<1e-9], c=np.ones(3)/3, shading={"wireframe": True})
# p.add_edges(V_msh, Geometry.f2e(T_msh), shading={"wireframe": True})

In [ ]:
p = mp.plot(V_msh_BFS, np.array([[32561, 13493, 17317, 13793]]), shading={"wireframe": True})
# Clipping
F_mask = np.zeros(F.shape[0])
xCoord = V[:, 0]
yCoord = V[:, 1]
zCoord = V[:, 2]
xThres_max = (xCoord.max() - xCoord.min()) * 0.12 + xCoord.min()
xThres_min = (xCoord.max() - xCoord.min()) * 0.05 + xCoord.min()
yThres_max = (yCoord.max() - yCoord.min()) * 0.3 + yCoord.min()
yThres_min = (yCoord.max() - yCoord.min()) * 0.25 + yCoord.min()
zThres_max = (zCoord.max() - zCoord.min()) * 0.85 + zCoord.min()
zThres_min = (zCoord.max() - zCoord.min()) * 0.6 + zCoord.min()
for i in range(int(F.shape[0]/4), int(F.shape[0]/4*3)):
    if ((V[F[i, 0], 0] < xThres_max) or (V[F[i, 1], 0] < xThres_max) or (V[F[i, 2], 0] < xThres_max)) and \
       ((V[F[i, 0], 1] < yThres_max) or (V[F[i, 1], 1] < yThres_max) or (V[F[i, 2], 1] < yThres_max)) and \
       ((V[F[i, 0], 0] > xThres_min) or (V[F[i, 1], 0] > xThres_min) or (V[F[i, 2], 0] > xThres_min)) and \
       ((V[F[i, 0], 1] > yThres_min) or (V[F[i, 1], 1] > yThres_min) or (V[F[i, 2], 1] > yThres_min)) and \
       ((V[F[i, 0], 2] > zThres_min) or (V[F[i, 1], 2] > zThres_min) or (V[F[i, 2], 2] > zThres_min)) and \
       ((V[F[i, 0], 2] < zThres_max) or (V[F[i, 1], 2] < zThres_max) or (V[F[i, 2], 2] < zThres_max)):
        F_mask[i] = True
F_mask = np.array(F_mask, dtype=bool)

colors = np.array([[1, 2, 3, 4]])
colors = np.tile(colors, (int(F.shape[0]/4), 1))
colors = colors.flatten('F')
p.add_mesh(V, F[F_mask, :], c=colors[F_mask], shading={"wireframe": True})

In [ ]:
# boundary_facets
F_bound = igl.boundary_facets(T_msh)
components = igl.face_components(F_bound)

plt = mp.plot(V_msh, F_bound[components == 0, :], shading={"wireframe": True})
# plt.add_edges(V, Geometry.f2e(F))

In [ ]:
# boundary_facets (clip)
F_bound = igl.boundary_facets(T_msh)
components = igl.face_components(F_bound)

N = F_bound.shape[0]
mask = np.zeros((N))
xCoord = V_msh[:, 0]
xThres = (xCoord.max() + xCoord.min()) / 2.0
for i in range(N):
    x_mean = (V_msh[F_bound[i, 0], 0] + V_msh[F_bound[i, 1], 0] + V_msh[F_bound[i, 2], 0]) / 3.0
    mask[i] = (x_mean < xThres) & (components[i] < 57)
mask = np.array(mask, dtype=bool)

plt = mp.plot(V_msh, F_bound[mask, :], shading={"wireframe": True})


In [ ]:
# boundary_facets
T_msh_temp = T_msh[(label == 8) | (label == 9), :]
Visual.Explode(V_msh, T_msh_temp, 0.25)
#plt = mp.plot(V_msh, F_bound[c == 0, :], shading={"wireframe": True})
#plt.add_mesh(V_msh, F_bound[c == 1, :])

In [ ]:
F_bound[components == 2, :]

In [ ]:
testArr = np.array([5787, 81, 52, 5788])
M = T_msh.shape[0]
for i in range(M):
    if (T_msh[i, 1] in testArr) and (T_msh[i, 2] in testArr) and (T_msh[i, 3] in testArr) :
        print(i)
        print(T_msh[i, :])
        print(label[i])
        break
print("done")